In [1]:
import numpy as np
import tensorflow as tk
import matplotlib.pyplot as plt
import os
import cv2

import mediapipe as mp
import shutil
import csv
            
import tkinter as tk
from tkinter import filedialog

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

"""
    List of classes (e.g. 'A' or 'C') in dataset directory
"""
def getClasses(file_path):
    classes = []
    for file in os.listdir(file_path):
        if os.path.isdir(file_path + '/' + file):
            classes.append(file)
    return classes


"""
    List of image files in path
"""
def getImageFiles(path):
    image_files = []
    for file in os.listdir(path):
        if (file.endswith('.png') or file.endswith('.jpg')):
            image_files.append(file)  
    return image_files


"""
    This function copies every file that was selected to the output to insert it in the dataset
    sel_files: List of files which were selected to be copied to the output path
    folder: Class (e.g 'A' or 'C')
    in_path: Path were images are stored (e.g. 'Dataset1/A/')
    out_path: Out Path (e.g. 'Output/Dataset1/')
    anootated_img: create imgs with drawn skeleton?

    copy_img: copy images to destination directory?
"""
def saveDataset(sel_files, folder, in_path, out_path, annotate_img=True, copy_img=True):
    if not os.path.exists(out_path + '/' + folder.upper()):
        os.mkdir(out_path + '/' + folder.upper())
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=1,
            min_detection_confidence=0.85) as hands:

        # Go through each picture of class
        for idx, file in enumerate(sel_files):

            ########################################################################################################
            # Preprocessing
            ########################################################################################################

            # flip input image so that a picture of a right hand is detected as a right hand 
            image = cv2.flip(cv2.imread(in_path + '/' + file),1)
            annotated_image = image.copy()
            # annotated_image is the ouput image with the drawn skeleton in it
            # cv2 reads image in BGR, image has to be converted to RGB (first tests brought conclusion that hand isn't detected in BGR spektrum)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

            ########################################################################################################
            # Feature Extraction
            ########################################################################################################

            # apply MediaPipe Hands to the picture
            results = hands.process(image)
            # Print handedness and draw hand landmarks on the image.
            #print('Handedness:', results.multi_handedness)
            if not results.multi_hand_landmarks:
                print(file + ': No hand detected. Discard file!')
                os.remove(in_path + '/' + file)
                continue
            
            if copy_img:
                shutil.copyfile(in_path + '/' + file, out_path + '/' + folder.upper() +'/' + file)
            
            if gen_csv:
                for i in range(21):
                    lm = np.array([hand_landmarks.landmark[i].x,hand_landmarks.landmark[i].y,hand_landmarks.landmark[i].z], dtype=float)
                    coord[i,:] = lm
            if annotate_img:
                if not os.path.exists(out_path + '/Annotated'): 
                    os.mkdir(out_path + '/Annotated')
                if not os.path.exists(out_path + '/Annotated/' + folder.upper()):
                    os.mkdir(out_path + '/Annotated/' + folder.upper())                    
                for hand_landmarks in results.multi_hand_landmarks:
                    # print('hand_landmarks:', hand_landmarks)
                    mp_drawing.draw_landmarks(
                            annotated_image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())     
                    cv2.imwrite(out_path + '/Annotated/' + folder.upper() + '/' + file, cv2.flip(annotated_image,1))     
        ###########    
        
"""
    This function generates a list of selected files which will be used in the dataset
    If amount of files to be selected (sel_amount) exceeds numer of available files (amount of sel_files), every file is used for dataset
    Elsewise sel_amount files get selected randomly from available files to create dataset
    
    sel_files: List of all files in which a hand was successfully detected
    sel_amount: Amount of files taken from sel_files
"""    
def createDatasetImages(sel_files,sel_amount=30, sel_all=False):
    # Creat class in output folder

    copy_files = []
    
    amount_filtered = len(sel_files)
    
    if (amount_filtered > 0):
        if sel_all:
            copy_files = sel_files
            sel_amount = amount_filtered
        else:
            if sel_amount > amount_filtered:
                copy_files = sel_files
                sel_amount = amount_filtered
                
            for i in range(sel_amount):
                idx = np.random.randint(len(sel_files))
                #print('Copy File ' + SEL_FILES[idx])

                copy_files.append(sel_files[idx])
                del sel_files[idx]        
    return copy_files  


"""
    Create a list of all files in class directory in which a hand is detected
    delete_img: delete non skeletoin images in origin directory?
"""
def scanRawData(image_files, in_path, delete_img=False):
    sel_files = []
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.85) as hands:

        # Go through each picture of class
        #################
        for idx, file in enumerate(image_files):
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).

            ########################################################################################################
            # Preprocessing
            ########################################################################################################

            # flip input image so that a picture of a right hand is detected as a right hand 
            image = cv2.flip(cv2.imread(in_path + '/' +file),1)
            # annotated_image is the ouput image with the drawn skeleton in it
            # cv2 reads image in BGR, image has to be converted to RGB (first tests brought conclusion that hand isn't detected in BGR spektrum)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

            ########################################################################################################
            # Feature Extraction
            ########################################################################################################

            # apply MediaPipe Hands to the picture
            results = hands.process(image)
            # Print handedness and draw hand landmarks on the image.
            #print('Handedness:', results.multi_handedness)
            if not results.multi_hand_landmarks or 'depth' in file:
                if delete_img:
                    os.remove(in_path + '/' + file)
                continue
            sel_files.append(file)
        #################
    return sel_files
# Process all directories selected


In [5]:
root = tk.Tk()
root.withdraw()

IMAGE_FILES = []
SEL_FILES = []
CLASSES = []

#ALL = 1000
#SEL_ALL = False


ANNOTATE = False
COPY = True
DELETE = False
SELECT_ALL = False

"""
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_A',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_B'
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_C',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_D',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_E',
            'C:/Users/gianl/Desktop/Dataset_MOD/Dataset ASL Hand Gestures/original_frames',
            'C:/Users/gianl/Desktop/Dataset_MOD/Kaggle_3/kaggle_3_asl_alphabet_train',
            'C:/Users/gianl/Desktop/Dataset_MOD/Kaggle_2/asl_alphabet_train/kaggle_2_asl_alphabet_train',
            'C:/Users/gianl/Desktop/Dataset_MOD/New_Data_1',
            'C:/Users/gianl/Desktop/Dataset_MOD/Video'
            

 
"""


# Copy here all paths to the raw data sets to be used in the program run
# For each data set in the SEL_AMOUNT array the amount of pictures to be selected has to be specified
PATHS = [   'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_A',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_B',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_C',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_D',
            'C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_E',
            'C:/Users/gianl/Desktop/Dataset_MOD/Dataset ASL Hand Gestures/original_frames',
            'C:/Users/gianl/Desktop/Dataset_MOD/Kaggle_3/kaggle_3_asl_alphabet_train',
            'C:/Users/gianl/Desktop/Dataset_MOD/Kaggle_2/asl_alphabet_train/kaggle_2_asl_alphabet_train',
            'C:/Users/gianl/Desktop/Dataset_MOD/New_Data_1',
            'C:/Users/gianl/Desktop/Dataset_MOD/Video']



SEL_AMOUNT = [18,18,18,18,18,10,70,40,20,25]


# Select output directory
output_path = filedialog.askdirectory(title="Select output")




for idxP, file_path in enumerate(PATHS):
    print('Input: ' + file_path)
    temp_output = output_path + '/' + os.path.basename(os.path.normpath(file_path+'/'))
    print('Output: ' + temp_output)
    if not os.path.exists(temp_output):
        os.mkdir(temp_output) 
    
    # Get every class available in Input directory (Folder names are used to creat for example A - Z)
    CLASSES = getClasses(file_path)
    
    # Go through every class
    for idxC, folder in enumerate(CLASSES):   
        sub_path = os.path.join(file_path + '/' + folder)
        
        #### Collect all files in directory
        IMAGE_FILES = getImageFiles(sub_path) 
        amount = len(IMAGE_FILES) ## All image files
        print('*****************************************************************')
        print(folder)
        print('Amount of Files: ' + str(amount))  
        print('*****************************************************************')

        #### get List of Files with skeleton in it
        SEL_FILES = scanRawData(IMAGE_FILES,sub_path, delete_img=DELETE) 
        amount_filtered = len(SEL_FILES) ## Files in which a hand was detected
        print(str(amount_filtered) + '/' + str(amount) + ' files with hands detected')
        #print(SEL_FILES)
        #### Select specified amount of files to create dataset and copy them to output
        export_files = createDatasetImages(SEL_FILES, sel_amount=SEL_AMOUNT[idxP],sel_all=SELECT_ALL)            

        print(str(len(export_files)) + ' files selected to copy')
        #print(export_files)
        
        saveDataset(export_files, folder, sub_path, temp_output, annotate_img=ANNOTATE, copy_img=COPY)
            
            
    print('*****************************************************************')
    print('*****************************************************************')
print("Process completed!")


Input: C:/Users/gianl/Desktop/Dataset_MOD/fingerspelling5//dataset5/fingerspelling_A
Output: C:/Users/gianl/Desktop/out_2/fingerspelling_A
*****************************************************************
a
Amount of Files: 10
*****************************************************************
10/10 files with hands detected
10 files selected to copy
*****************************************************************
b
Amount of Files: 126
*****************************************************************
126/126 files with hands detected
18 files selected to copy
*****************************************************************
c
Amount of Files: 29
*****************************************************************
29/29 files with hands detected
18 files selected to copy
*****************************************************************
d
Amount of Files: 81
*****************************************************************
81/81 files with hands detected
18 files selected to copy
**********

358/358 files with hands detected
18 files selected to copy
*****************************************************************
o
Amount of Files: 344
*****************************************************************
344/344 files with hands detected
18 files selected to copy
*****************************************************************
p
Amount of Files: 127
*****************************************************************
127/127 files with hands detected
18 files selected to copy
*****************************************************************
q
Amount of Files: 429
*****************************************************************
429/429 files with hands detected
18 files selected to copy
*****************************************************************
r
Amount of Files: 415
*****************************************************************
415/415 files with hands detected
18 files selected to copy
*****************************************************************
s
Amount of Fi

68/68 files with hands detected
18 files selected to copy
*****************************************************************
b
Amount of Files: 7
*****************************************************************
7/7 files with hands detected
7 files selected to copy
*****************************************************************
c
Amount of Files: 257
*****************************************************************
257/257 files with hands detected
18 files selected to copy
*****************************************************************
d
Amount of Files: 308
*****************************************************************
308/308 files with hands detected
18 files selected to copy
*****************************************************************
e
Amount of Files: 109
*****************************************************************
109/109 files with hands detected
18 files selected to copy
*****************************************************************
f
Amount of Files: 233


*****************************************************************
p
Amount of Files: 193
*****************************************************************
193/193 files with hands detected
18 files selected to copy
*****************************************************************
q
Amount of Files: 107
*****************************************************************
107/107 files with hands detected
18 files selected to copy
*****************************************************************
r
Amount of Files: 293
*****************************************************************
293/293 files with hands detected
18 files selected to copy
*****************************************************************
s
Amount of Files: 120
*****************************************************************
120/120 files with hands detected
18 files selected to copy
*****************************************************************
t
Amount of Files: 120
***************************************************

*****************************************************************
C
Amount of Files: 406
*****************************************************************
406/406 files with hands detected
70 files selected to copy
*****************************************************************
D
Amount of Files: 332
*****************************************************************
332/332 files with hands detected
70 files selected to copy
*****************************************************************
E
Amount of Files: 512
*****************************************************************
512/512 files with hands detected
70 files selected to copy
*****************************************************************
F
Amount of Files: 791
*****************************************************************
791/791 files with hands detected
70 files selected to copy
*****************************************************************
G
Amount of Files: 510
***************************************************

1888/1888 files with hands detected
40 files selected to copy
*****************************************************************
U
Amount of Files: 2271
*****************************************************************
2271/2271 files with hands detected
40 files selected to copy
*****************************************************************
V
Amount of Files: 2335
*****************************************************************
2335/2335 files with hands detected
40 files selected to copy
*****************************************************************
W
Amount of Files: 2195
*****************************************************************
2195/2195 files with hands detected
40 files selected to copy
*****************************************************************
X
Amount of Files: 1810
*****************************************************************
1810/1810 files with hands detected
40 files selected to copy
*****************************************************************


21/21 files with hands detected
21 files selected to copy
*****************************************************************
M
Amount of Files: 20
*****************************************************************
20/20 files with hands detected
20 files selected to copy
*****************************************************************
N
Amount of Files: 19
*****************************************************************
19/19 files with hands detected
19 files selected to copy
*****************************************************************
O
Amount of Files: 20
*****************************************************************
20/20 files with hands detected
20 files selected to copy
*****************************************************************
P
Amount of Files: 11
*****************************************************************
11/11 files with hands detected
11 files selected to copy
*****************************************************************
Q
Amount of Files: 15
******

In [ ]:
### to .csv